# PRACTICAL SESSION 2:
Simulating truncated homogeneous sphere gravitational collapse

## MODULES IMPORT

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy import constants as const
from astropy import units as u

In [2]:
# Chosen units (same name as in astropy https://docs.astropy.org/en/stable/units/#module-astropy.units.astrophys)

my_u = ["R_sun","yr","M_sun"]

# Physical constants converted to chosen units

G = const.G.to(my_u[0] + "^3/(" + my_u[1] + "^2 " + my_u[2] + ")").value
c = const.c.to(my_u[0] + "/" + my_u[1]).value

# Conversion constants between physical and internal units

R0 = 1.
M0 = 1.
T0 = np.sqrt(R0**3/(G*M0))

# 1 - INPUT GENERATION:
Sampling N p.cles 

In [3]:
# Setting the random number generator

rng = np.random.default_rng()

In [4]:
N = 100 # number of p.cles
M = 1   # total mass (in units of solar mass)
R = 1   # radius (in units of solar radius)

phi = 2 * np.pi * rng.uniform(size=N)
the = np.arccos(1 - 2 * rng.uniform(size=N))
rho = R * rng.uniform(size=N)**(1/3)

x = rho * np.sin(the) * np.cos(phi)
y = rho * np.sin(the) * np.sin(phi)
z = rho * np.cos(the)

In [5]:
file_in = 'IF_Sun_100.txt'

m = M / M0 / N * np.ones(N) # mass of a single p.cle
v = np.zeros(N)             # p.cles at rest

x = x / R0 # initial position
y = y / R0
z = z / R0

time_in = 0 # initial time

output = np.column_stack((m, x, y, z, v, v, v))
with open(file_in, 'w', encoding="utf-8") as file:
    file.write(f'{N}\n')
    file.write(f'{time_in}\n')
    np.savetxt(file, output)

# 2 - DATA ANALYSIS

In [6]:
file_out = 'OF_Sun_2.txt'

n = N + 2 # 'block' length
m = 2     # number of rows to skip (i.e. lines containing n. bodies and time)

time_rows = []
data_rows = []
with open(file_out, "r") as file:
    lines = file.readlines()
    for i in range(0, len(lines), n):  # move through rows every n rows
        time_rows.extend(lines[i+1:i+m])
        data_rows.extend(lines[i+m:i+n])

time = np.loadtxt(time_rows)
data = np.loadtxt(data_rows)

ValueError: the number of columns changed from 7 to 1 at row 3; use `usecols` to select a subset and avoid this error

In [ ]:
# Fetching data

m = data[:, 0] # masses

r = data[:, 1:4] # positions
d = np.linalg.norm(r, axis=1)

v = data[:, 4:7] # velocities

d.shape

In [ ]:
fig, ax = plt.subplots(1, 1, layout='constrained')

for i in range(N):
    ax.plot(time, d[i::N], '.')